In [ ]:
import json
import glob
import string
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.feature import match_template, canny
from skimage.filters import threshold_otsu, rank, sobel
from skimage.morphology import disk, watershed
from scipy import ndimage as ndi
%matplotlib inline

In [ ]:
data_dir = 'data/'
template_dir = 'templates/'
moon_dir = data_dir+'moon/'
moon_template_dir = template_dir+'moon/'

In [ ]:
moon_images = glob.glob(moon_dir+'*.jpg')

In [ ]:
moon_template_images = glob.glob(moon_template_dir+'[0-9]*.jpg')

In [ ]:
def match_images(filenames, templates, threshold=0.5):
    matches = {template: [] for template in templates}
    
    template_images = []
    for template in templates:
        template_images.append(io.imread(template))

    for filename in filenames:
        match_results = []
        image = io.imread(filename, as_grey=True)
        for template in template_images:
            match_results.append(match_template(image, template))
        
        # Find the template with the greatest match.
        match_results = np.array(match_results)
        max_idx = np.unravel_index(match_results.argmax(), match_results.shape)
        if match_results[max_idx] > threshold:
            template_match = templates[max_idx[0]]
            image_id = filename.translate(None, string.letters).translate(None, string.punctuation)
            matches[template_match].append(image_id)
    
    return matches

In [ ]:
moon_template_matches = match_images(moon_images, moon_template_images)

In [ ]:
sample_moon = io.imread(moon_dir+'121.jpg', as_grey=True)

In [ ]:
io.imshow(sample_moon)

In [ ]:
plt.hist(sample_moon.ravel(), bins=20,  fc='k')

In [ ]:
io.imshow(sample_moon > 0.01)

In [ ]:
local_otsu_moon = rank.otsu(sample_moon, disk(10))

In [ ]:
io.imshow(sample_moon < local_otsu_moon)

In [ ]:
global_otsu_moon = threshold_otsu(sample_moon)

In [ ]:
io.imshow(sample_moon >= global_otsu_moon)

In [ ]:
moon_edges = canny(sample_moon, sigma=0.5)

In [ ]:
io.imshow(moon_edges)

In [ ]:
moon_filled = ndi.binary_fill_holes(moon_edges)

In [ ]:
io.imshow(moon_filled)

In [ ]:
elevation_map = sobel(sample_moon)

In [ ]:
io.imshow(elevation_map)

In [ ]:
segment_markers = np.zeros_like(sample_moon)

In [ ]:
segment_markers[sample_moon < 0.01] = 1
segment_markers[sample_moon > 0.2] = 2

In [ ]:
segmentation = watershed(elevation_map, segment_markers)

In [ ]:
io.imshow(segmentation)